#**MOUNTING GOOGLE DRIVE AND SETTING PATH**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/MyDrive/Tensorflow/workspace

In [ ]:
import os
os.environ['PYTHONPATH'] += '/content/gdrive/MyDrive/Tensorflow/workspace/:/content/gdrive/MyDrive/Tensorflow/workspace/models/:/content/gdrive/MyDrive/Tensorflow/workspace/models/research/:/content/gdrive/MyDrive/Tensorflow/workspace/models/research/slim'

#**INSTALLING RELEVANT LIBRARIES**

In [ ]:
!pip install tensorflow_io
!pip install tqdm
!pip install pydicom
!pip install ensemble-boxes
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

# **DOWNLOADING MODELS**

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models

# **COMPILING PROTO FILES**

In [ ]:
%cd /content/gdrive/MyDrive/Tensorflow/workspace/models/research
!protoc object_detection/protos/*.proto --python_out=.
%cd ..
%cd ..

# **Installing Coco Api for Evaluating the model**

In [ ]:
!pip install cython 
!git clone https://github.com/cocodataset/cocoapi.git
%cd /content/gdrive/My Drive/Tensorflow/cocoapi/PythonAPI
!make
%cd ..
%cd ..
!cp -r cocoapi/PythonAPI/pycocotools models/research/

# **Installing object detection api**

In [ ]:
%cd /content/gdrive/MyDrive/Tensorflow/workspace/models/research
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd ..
%cd ..

# **Importing Relevant libraries**

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from ensemble_boxes import *
from tqdm.notebook import tqdm

import pydicom
from pydicom.tag import Tag

import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_hub as hub
from tensorflow.keras import Sequential
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from object_detection.utils import label_map_util as map_util
from object_detection.utils import visualization_utils as viz_util
from object_detection.utils import ops as ops_util
from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
from object_detection.dataset_tools import tf_record_creation_util
from object_detection.utils import dataset_util
import contextlib2

import tensorflow as tf
from object_detection.utils import label_map_util as map_util
from object_detection.utils import visualization_utils as viz_util
from object_detection.utils import ops as ops_util
from object_detection.utils import config_util

import requests
import tarfile
from tqdm.notebook import tqdm
from io import BytesIO
from shutil import copy2
import random


from google.protobuf import text_format

# **Converting Dicom File to Np Array**

In [ ]:
def read_dicom(path, max_dim):
    image_bytes = tf.io.read_file(path)
    image = tfio.image.decode_dicom_image(
        image_bytes, 
        dtype = tf.uint16
    )
    
    image = tf.squeeze(image, axis = 0)
    
    h, w, _ = image.shape
    
    if max_dim != None:
        image = tf.image.resize(
            image, 
            (max_dim, max_dim), 
            preserve_aspect_ratio = True
        )

    image = image - tf.reduce_min(image)
    image = image / tf.reduce_max(image)
    image = tf.cast(image * 255, tf.uint8)
    
    return image, h, w

Visualizing how a sample image looks like

In [ ]:
# Read one image to check the loading is OK
%matplotlib inline
demo_image = "6d5acf3f8a973a26844d617fffe72998.dicom"

img,h,w = read_dicom(os.path.join(path, "train", demo_image),500)
print(img.numpy())
plt.figure(figsize = (10,10))
plt.imshow(tf.squeeze(img), 'gray')

# **Loading Annotation data in train.csv as a dataframe**

In [ ]:
# Create a dataframe containing the training data
csv_path = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv'
df = pd.read_csv(csv_path)
df

# **Visualizing Annotations**

In [ ]:
# method to plot images with its annotations
def plot_annotations(df_item, path, hide_axis = False):
    demo_image=df_item['image_id']+'.dicom'
    img,h,w = read_dicom(os.path.join(path,"train",demo_image ),None)
    img=img.numpy()
    # Convert the x-ray image into RGB
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    
    if (np.isnan(df_item["x_min"]) and np.isnan(df_item["y_min"])):
        return print("No detection found!")
    
    # Declare coordinates and convert them to integers
    x_min = int(df_item["x_min"])
    y_min = int(df_item["y_min"])
    x_max = int(df_item["x_max"])
    y_max = int(df_item["y_max"])

    # Create figure
    plt.figure(figsize = (10,10))
    
    # Create rectangle where the annotation is located
    image = cv2.rectangle(img=img,rec=(x_min,y_min,x_max-x_min,y_max-y_min), color = (0,255,0),thickness = 10)
    
    # Add label to the annotation
    image = cv2.putText(image, df_item["class_name"], (int(df_item["x_min"]),int(df_item["y_min"])-20), cv2.FONT_HERSHEY_TRIPLEX, 2, (0,0,0), 3)
                
    # Plot image
    plt.imshow(image)
    plt.axis("Off")
    plt.show()

In [ ]:
# method to plot images with its annotations
def plot_all_labels(df_item, path):
    img,h,w=read_dicom(os.path.join(path,"train", "{}.dicom".format(df_item.iloc[1]["image_id"])),None)
    img=img.numpy()
    # Convert the x-ray image into RGB
    image = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    
    # Create figure
    plt.figure(figsize = (10,10))
    plt.title("Image ID - {}".format(df_item.iloc[1]["image_id"]))
    
    for index,item in df_item.iterrows():
        
        if (np.isnan(item["x_min"]) and np.isnan(item["y_min"])):
            continue

        # Declare coordinates and convert them to integers
        x_min = int(item["x_min"])
        y_min = int(item["y_min"])
        x_max = int(item["x_max"])
        y_max = int(item["y_max"])

        # Create rectangle where the annotation is located
        image = cv2.rectangle(img=image,rec=(x_min,y_min,x_max-x_min,y_max-y_min), color = (0,255,0),thickness = 10)

        # Add label to the annotation
        image = cv2.putText(image, item["class_name"], (int(item["x_min"]),int(item["y_min"])), cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(0,0,0), thickness=3)

                
    # Plot image
    plt.imshow(image)
    plt.axis("Off")
    plt.show()

In [ ]:
plot_all_labels(image_annotations, path)

# **Visualizing and Preprocessing Data**

* ### Exploring distribution of radiologists

In [ ]:
temp = df[["image_id", "rad_id"]].drop_duplicates().reset_index(drop = True)
temp = temp.groupby(["rad_id"]).agg(
    count = pd.NamedAgg("image_id", "count")
).reset_index()
temp

In [ ]:
# Plot the 3rd dataset item

plot_annotations(df.iloc[5],path, True)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 5))

sns.countplot(
    df["rad_id"], 
    palette = "tab10", 
    order = list(temp["rad_id"]), 
    ax = ax[0]
)
ax[0].set_title("Number of annotations by radiologists")

sns.barplot(
    x = "rad_id", 
    y = "count", 
    data = temp, 
    palette = "tab10", 
    ax = ax[1]
)
ax[1].set_title("Number of x-rays seen by radiologists")

fig.show()

Radiologists 9, 10 and 8 saw most number of x-rays and made most annotations.

In [ ]:
sns.barplot(
    x = "class_name", 
    y = "count", 
    data = temp, 
    palette = "tab10"
)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
def CLAHE(image):
    clahe = cv2.createCLAHE(
        clipLimit = 2., 
        tileGridSize = (10, 10)
    )
    
    image = clahe.apply(image.numpy()) 
    image = np.expand_dims(image, axis = 2)
    
    return image

In [ ]:
%matplotlib inline

fig = plt.figure(figsize = (8, 8))

axes = fig.add_subplot(1, 2, 1)
plt.imshow(tf.squeeze(img), cmap = "gray")
axes.set_title("Original")

axes = fig.add_subplot(1, 2, 2)
image = CLAHE(img)
plt.imshow(tf.squeeze(img), cmap = "gray")
axes.set_title("Post CLAHE")

* ### Incrementing Class ids by 1 as our API requires classes for 1 to n

In [ ]:
df["class_id"] = df["class_id"] + 1 # Incrementing by 1
df

* ### 14 unique colors to annotate the abnormalities.

In [ ]:
LABEL_COLORS = [
    (230, 25, 75), (60, 180, 75), (255, 225, 25), (0, 130, 200), (245, 130, 48), (145, 30, 180), (70, 240, 240), 
    (240, 50, 230), (210, 245, 60), (250, 190, 212), (0, 128, 128), (220, 190, 255), (170, 110, 40), (255, 250, 200), 
]
LabelMap["colors"] = LABEL_COLORS
LabelMap

* ### Label map files have the extention .pbtxt so here we are saving LabelMap as .pbtxt file

In [ ]:
def save_mapping(LabelMap):
    msg = StringIntLabelMap()
    
    for i, row in LabelMap.iterrows():
        msg.item.append(StringIntLabelMapItem(id = row["class_id"], name = row["class_name"]))
    
    text = str(text_format.MessageToBytes(msg, as_utf8 = True), 'utf-8')
    
    f = open("LabelMap.pbtxt", "w")
    f.write(text)
    f.close()
    
save_mapping(LabelMap)

* ### Removing Examples with No Findings as it will not be used in training

In [ ]:
# Remove examples with no findings (won't be used for training)
df = df.dropna().reset_index(drop = True)
df

* ### TensorFlow requires a label map, which namely maps each of the used labels to an integer values.**

In [ ]:
LabelMap = df.loc[df["class_name"] != "No finding", ["class_name", "class_id"]] # Removing the examples with no finding
LabelMap = LabelMap.drop_duplicates().reset_index(drop = True)
LabelMap

* ### Changing Data Types

In [ ]:
df = df.astype({
    "x_min": int, 
    "y_min": int, 
    "x_max": int, 
    "y_max": int,
    "class_id": str
})

In [ ]:
df

In [ ]:
def plot_boxes(image, data, title):    
    img = cv2.cvtColor(image.numpy(), cv2.COLOR_GRAY2RGB)
    
    for i, row in data.iterrows():
    
        x1, y1 = row["x_min"], row["y_min"]
        x2, y2 = row["x_max"], row["y_max"]
    
        cv2.rectangle(
            img,
            pt1 = (x1, y1),
            pt2 = (x2, y2),
            color=row['colors'],
            thickness = 2
        )
    
        cv2.putText(
            img, 
            row["class_name"], 
            (x1, y1-5), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            row['colors'], 
            1
        )

    plt.figure(figsize = (8, 8))
    plt.imshow(img) 
    plt.title(title)

In [ ]:
# Selecting a particular radiologist
demo_rad = "R9"

# Preprocessing metadata to suit needs
data = df.loc[
    (df["image_id"] == demo_image[:-6]) & (df["rad_id"] == demo_rad),
    ["class_name", "x_min", "y_min", "x_max", "y_max"]
]

H, W, _ = img.shape
data[["x_min", "x_max"]] = (data[["x_min", "x_max"]]* W/w).astype(int)
data[["y_min", "y_max"]] = (data[["y_min", "y_max"]]* H/h).astype(int)

data = pd.merge(data, LabelMap)

# Plotting annotation by radiologist
plot_boxes(img, data, "Labels for " + demo_image + " by " + demo_rad)

In [ ]:
# Preprocessing metadata to suit needs
data = df.loc[
    (df["image_id"] == demo_image[:-6]),
    ["class_name", "x_min", "y_min", "x_max", "y_max"]
]

H, W, _ = img.shape
data[["x_min", "x_max"]] = (data[["x_min", "x_max"]]* W/w).astype(int)
data[["y_min", "y_max"]] = (data[["y_min", "y_max"]]* H/h).astype(int)

data = pd.merge(data, LabelMap)

# Plotting annotation by all radiologists
plot_boxes(img, data, "Labels for " + demo_image + " by all radiologists")

In [ ]:
# Preprocessing metadata to suit needs
data = df.loc[
    (df["image_id"] == demo_image[:-6]),
    ["class_name", "x_min", "y_min", "x_max", "y_max"]
]

H, W, _ = img.shape
data[["x_min", "x_max"]] = (data[["x_min", "x_max"]]* W/w).astype(int)
data[["y_min", "y_max"]] = (data[["y_min", "y_max"]]* H/h).astype(int)

data = pd.merge(data, LabelMap)

# Plotting annotation by all radiologists
plot_boxes(img, data, "Labels for " + demo_image + " by all radiologists")

In [ ]:
print(w)
print(W)

In [ ]:
print(w)
print(W)

We need not train our model on multiple annotations of the same abnormality. We shall use a technique called Weighted Boxes Fusion (WBF) to provide us with the best annotation. This will definitely reduce the metadata size by a lot.

* ### Preprocessing as needed for Weighted Boxes Fusion (WBF)

In [ ]:
data = df.loc[
    (df["image_id"] == demo_image[:-6]),
    ["class_name", "x_min", "y_min", "x_max", "y_max"]
]

data[["x_min", "x_max"]] = data[["x_min", "x_max"]]/w
data[["y_min", "y_max"]] = data[["y_min", "y_max"]]/h

data = pd.merge(data, LabelMap)

boxes_list = data[["x_min", "y_min", "x_max", "y_max"]].values.tolist()
scores_list = [1]*len(boxes_list)
labels_list = list(data["class_id"])

* ### Applying Weighted Box Fusion

In [ ]:
boxes, _, labels = weighted_boxes_fusion(
    boxes_list = [boxes_list],
    scores_list = [scores_list],
    labels_list = [labels_list],
    weights = None, 
    iou_thr = 0.3, 
    skip_box_thr = 0.0001
)

data = pd.DataFrame(boxes, columns = ["x_min", "y_min", "x_max", "y_max"])
data

* ### Postprocessing after applying WBF 

In [ ]:
data = pd.DataFrame(boxes, columns = ["x_min", "y_min", "x_max", "y_max"])

H, W,_= img.shape
data[["x_min", "x_max"]] = (data[["x_min", "x_max"]]* W).astype(int)
data[["y_min", "y_max"]] = (data[["y_min", "y_max"]]* H).astype(int)

data["class_id"] = labels.astype(int)

data = pd.merge(data,LabelMap)

# Plotting annotation by all radiologists
plot_boxes(img, data, "Labels for " + demo_image + " post WBF")

We have eliminated multiple annotations for the same abnormality.

In [ ]:
# Dropping rad_id as it is not required for training
df = df.drop(columns = ["rad_id"])

# Obtaining set of x-rays with at least one finding
xrays = set(df["image_id"]) # Only 4394 x-rays, not 15000. Roughly 30% of the x-rays remain.

In [ ]:
df

In [ ]:
dimensions = []
for i, xray in tqdm(enumerate(xrays)):
    ds = pydicom.dcmread(
        os.path.join(path, "train", xray + ".dicom"), 
        specific_tags = [
            Tag("0028", "0010"), # Tag for Rows (Height)
            Tag("0028", "0011")  # Tag for Columns (Width)
        ]
    )
    
    dimensions.append([xray, ds.Rows, ds.Columns])

dimensions

In [ ]:
dimensions = pd.DataFrame(dimensions, columns = ["image_id", "height", "width"])
df = pd.merge(dimensions, df)
df

* ### Normalizing coordinates

In [ ]:
df["x_min"], df["x_max"] = df["x_min"]/df["width"], df["x_max"]/df["width"]
df["y_min"], df["y_max"] = df["y_min"]/df["height"], df["y_max"]/df["height"]
df

In [ ]:
df["x_min"], df["x_max"] = df["x_min"]/df["width"], df["x_max"]/df["width"]
df["y_min"], df["y_max"] = df["y_min"]/df["height"], df["y_max"]/df["height"]
df

* ### Applying WBF on complete dataframe

In [ ]:
# Before applying WBF we had 36096 rows
df_list = []
for i, xray in tqdm(enumerate(xrays)):
    data = df[df["image_id"] == xray]

    boxes_list = data[["x_min", "y_min", "x_max", "y_max"]].values.tolist()
    scores_list = [1]*len(boxes_list)
    labels_list = list(data["class_id"])

    # Applying WBF
    boxes, _, labels = weighted_boxes_fusion(
        boxes_list = [boxes_list],
        scores_list = [scores_list],
        labels_list = [labels_list],
        weights = None, 
        iou_thr = 0.3, 
        skip_box_thr = 0.0001
    )
    
    data = pd.DataFrame(boxes, columns = ["x_min", "y_min", "x_max", "y_max"]) 
    # Leaving the coordinates normalized since the API expects them to be so. 
    
    data["class_id"] = labels.astype(int)
    
    data["image_id"] = xray 
    
    df_list.append(data)

In [ ]:
df

In [ ]:
df = pd.concat(df_list) # After applying WBF we have 21836 rows
df

In [ ]:
df = pd.merge(df, LabelMap)
df = df.drop(columns=['colors'])
df

* ### Stratified K-Fold Sharding

In [ ]:
num_shards = 25

skf = StratifiedKFold(
    n_splits = num_shards, 
    shuffle = True, 
    random_state = 0
)

df_folds = df[['image_id']].copy()

df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()   # Number of bounding boxes in the image
df_folds.loc[:, 'object_count'] = df.groupby('image_id')['class_id'].nunique() # Number of classes in the image
df_folds

* ### Preparing stratify groups

In [ ]:

df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['object_count'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)
df_folds

* ### Determining which fold the x-ray will fall in

In [ ]:
df_folds.loc[:, 'fold'] = 0
skf_split = skf.split(
    X = df_folds.index, 
    y = df_folds['stratify_group']
)

for fold_number, (train_index, val_index) in enumerate(skf_split):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number
    
df_folds.reset_index(inplace = True)

df_folds

In [ ]:
df = pd.merge(df, df_folds)

temp = df.groupby(["fold", "class_name"]).agg(
    count = pd.NamedAgg("class_name", "count")
).reset_index()

temp = temp.pivot_table(
    index = "class_name",
    columns = "fold",
    values = "count"
)

In [ ]:
df

# **Converting our annotations into the so called TFRecord format.**

In [ ]:
def create_tf_record(img_path, max_dim, img_df):
    
    filename = img_path.split("/")[-1].encode()
    source_id = img_path.encode()
    
    # Preprocess image 
    img, _, _ = read_dicom(img_path, max_dim)
    height, width, _ = img.shape
    img = CLAHE(img)
    
    # Encode as JPEG (Lossy compression)
    img = tf.io.encode_jpeg(
        img, 
        quality = 100, 
        format = 'grayscale'
    )
    
    img_bytes = img.numpy()
    
    img_format = b'jpeg'

    xmin_list = list(img_df["x_min"])
    xmax_list = list(img_df["x_max"])
    ymin_list = list(img_df["y_min"])
    ymax_list = list(img_df["y_max"])
    
    class_name_list = list(img_df["class_name"])
    class_name_list = [c.encode() for c in class_name_list]
    
    class_id_list = list(img_df["class_id"])
    
    # Creating TFRecord
    tf_record = tf.train.Example(
        features = tf.train.Features(
            feature = {
                'image/height': dataset_util.int64_feature(height),
                'image/width': dataset_util.int64_feature(width),
                'image/filename': dataset_util.bytes_feature(filename),
                'image/source_id': dataset_util.bytes_feature(source_id),
                'image/encoded': dataset_util.bytes_feature(img_bytes),
                'image/format': dataset_util.bytes_feature(img_format),
                'image/object/bbox/xmin': dataset_util.float_list_feature(xmin_list),
                'image/object/bbox/xmax': dataset_util.float_list_feature(xmax_list),
                'image/object/bbox/ymin': dataset_util.float_list_feature(ymin_list),
                'image/object/bbox/ymax': dataset_util.float_list_feature(ymax_list),
                'image/object/class/text': dataset_util.bytes_list_feature(class_name_list),
                'image/object/class/label': dataset_util.int64_list_feature(class_id_list),
            }
        )
    )
    
    return tf_record

In [ ]:
annot_path = "workspace/annotations" 
os.makedirs(annot_path, exist_ok = True) 

In [ ]:
img_cnt = np.zeros(num_shards, dtype = int)

max_dim=500

with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
        tf_record_close_stack, 
        annot_path, 
        num_shards
    )
    
    for i in tqdm(range(num_shards)):
        df_shard = df[df["fold"] == i]
        xrays = set(df_shard["image_id"])
        
        for xray in xrays:
            df_image = df_shard[df_shard["image_id"] == xray]
            
            img_path = os.path.join(path, "train", xray + ".dicom")
            tf_record = create_tf_record(img_path, max_dim, df_image)
            output_tfrecords[i].write(tf_record.SerializeToString())
            
            img_cnt[i] += 1

print("Converted {} images".format(np.sum(img_cnt)))
print("Images per shard: {}".format(img_cnt))

In [ ]:
df

In [ ]:
# Save dataframe
df.to_csv("data.csv", index = False)

#**Training and Validating**

In [ ]:
from shutil import copy2
# Creating workspace
os.makedirs("/content/gdrive/MyDrive/Tensorflow/workspace/pretrained_models", exist_ok = True)
os.makedirs("/content/gdrive/MyDrive/Tensorflow/workspace/models", exist_ok = True)
os.makedirs("/content/gdrive/MyDrive/Tensorflow/workspace/exported_models", exist_ok = True)
copy2("/content/gdrive/MyDrive/Tensorflow/workspace/models/research/object_detection/model_main_tf2.py", "/content/gdrive/MyDrive/Tensorflow/workspace")
copy2("/content/gdrive/MyDrive/Tensorflow/workspace/models/research/object_detection/exporter_main_v2.py", "/content/gdrive/MyDrive/Tensorflow/workspace")

In [ ]:
path_annot="/content/gdrive/MyDrive/Tensorflow/workspace/annotations"
raw_dataset = tf.data.TFRecordDataset(os.path.join(path_annot, "annotations-00000-of-00025"))

for raw_record in raw_dataset.take(1): # Select one shard from the TFRecords dataset
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
print(example)

In [ ]:
def GetData(example):
    xmin = example.features.feature['image/object/bbox/xmin'].float_list.value
    xmax = example.features.feature['image/object/bbox/xmax'].float_list.value
    ymin = example.features.feature['image/object/bbox/ymin'].float_list.value
    ymax = example.features.feature['image/object/bbox/ymax'].float_list.value

    class_name_list = example.features.feature['image/object/class/text'].bytes_list.value
    class_name_list = [c.decode() for c in class_name_list]

    class_id_list = example.features.feature['image/object/class/label'].int64_list.value

    data = pd.DataFrame(
        zip(xmin, ymin, xmax, ymax, class_name_list, class_id_list), 
        columns = ["x_min", "y_min", "x_max", "y_max", "class_name", "class_id"]
    )
    print(data)

    height = example.features.feature['image/height'].int64_list.value[0]
    width = example.features.feature['image/width'].int64_list.value[0]

    data[["x_min", "x_max"]] = (data[["x_min", "x_max"]]*width).astype(int)
    data[["y_min", "y_max"]] = (data[["y_min", "y_max"]]*height).astype(int)

    LABEL_COLORS = [
        (230, 25, 75), (60, 180, 75), (255, 225, 25), (0, 130, 200), (245, 130, 48), (145, 30, 180), (70, 240, 240), 
        (240, 50, 230), (210, 245, 60), (250, 190, 212), (0, 128, 128), (220, 190, 255), (170, 110, 40), (255, 250, 200) 
    ]
    data["colors"] = data["class_id"].apply(lambda x: LABEL_COLORS[x-1])
    
    
    img_encoded = example.features.feature['image/encoded'].bytes_list.value[0]
    image = tf.io.decode_jpeg(img_encoded)
    
    return data, image

In [ ]:
data, image = GetData(example)

In [ ]:
def plot_boxes(image, data, title):    
    img = cv2.cvtColor(image.numpy(), cv2.COLOR_GRAY2RGB)
    
    for i, row in data.iterrows():
    
        x1, y1 = row["x_min"], row["y_min"]
        x2, y2 = row["x_max"], row["y_max"]
    
        cv2.rectangle(
            img,
            pt1 = (x1, y1),
            pt2 = (x2, y2),
            color=row['colors'],
            thickness = 2
        )
    
        cv2.putText(
            img, 
            row["class_name"], 
            (x1, y1-5), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            row['colors'], 
            1
        )
    %matplotlib inline
    plt.figure(figsize = (8, 8))
    plt.imshow(img)
    plt.title(title)

In [ ]:
plot_boxes(image, data, "Image extracted from TFRecord")

#**Setting things for Efficientdet D1**

In [ ]:
path = "/content/gdrive/MyDrive/Tensorflow/workspace/pretrained_models"
fname = "pipeline.config"
model_name = "efficientdet_d1_coco17_tpu-32-300"
src = "/content/gdrive/MyDrive/Tensorflow/workspace/pretrained_models/efficientdet_d1_coco17_tpu-32/pipeline.config"
dst = "/content/gdrive/MyDrive/Tensorflow/workspace/models/efficientdet_d1_coco17_tpu-32-300/"
path_label = "/content/gdrive/MyDrive/Tensorflow/workspace/LabelMap.pbtxt" 
annot = "/content/gdrive/MyDrive/Tensorflow/workspace/annotations"
fpath = os.path.join("/content/gdrive/MyDrive/Tensorflow/workspace/models/efficientdet_d1_coco17_tpu-32-300", fname)
LabelMap = map_util.create_category_index_from_labelmap(
    path_label, 
    use_display_name = True
)

#**Downloading Efficientdet D1**

In [ ]:
# Download EfficientDet from Model Zoo
url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz"

r = requests.get(url)

# Extract model
thetarfile = tarfile.open(
    fileobj = BytesIO(r.content), 
    mode = "r|gz"
)

# Save model
thetarfile.extractall(path = path)

In [ ]:
# Moving pipeline.config file to models directory

os.makedirs(dst, exist_ok = True)

copy2(src, dst)

In [ ]:
len(LabelMap)

In [ ]:
os.makedirs(annot, exist_ok = True) 

In [ ]:
import random
annot_dir = os.listdir(annot)
random.Random(5).shuffle(annot_dir)
print(len(annot_dir))

In [ ]:
train_data = annot_dir[:-2]
train_data = [os.path.join(annot, d) for d in train_data]

valid_data = annot_dir[-2:]
valid_data = [os.path.join(annot, d) for d in valid_data]

#test_data = annot_dir[-2:]
#test_data = [os.path.join(annot, d) for d in test_data]

In [ ]:
len(train_data)

In [ ]:
valid_data

#**Editing Pipeline.config file of Efficientdet d1**

In [ ]:
# Making recommended changes
config_dic = config_util.get_configs_from_pipeline_file(fpath)

config_dic["model"].ssd.num_classes = len(LabelMap)
config_dic["model"].ssd.image_resizer.keep_aspect_ratio_resizer.min_dimension = 300
config_dic["model"].ssd.image_resizer.keep_aspect_ratio_resizer.max_dimension = 512

config_dic["train_config"].batch_size = 8
config_dic["train_config"].fine_tune_checkpoint = os.path.join("/content/gdrive/MyDrive/Tensorflow/workspace/pretrained_models","efficientdet_d1_coco17_tpu-32", "checkpoint/ckpt-0")
config_dic["train_config"].fine_tune_checkpoint_type = "detection"
config_dic["train_config"].use_bfloat16 = False # Set to True if training on a TPU
config_dic["train_config"].num_steps = 5000

config_dic["train_input_config"].label_map_path = path_label
config_dic["train_input_config"].tf_record_input_reader.input_path[:] = train_data

config_dic["eval_input_configs"][0].label_map_path = path_label
config_dic["eval_input_configs"][0].tf_record_input_reader.input_path[:] = valid_data

In [ ]:
config = config_util.create_pipeline_proto_from_configs(config_dic)
config_util.save_pipeline_config(config, "/content/gdrive/MyDrive/Tensorflow/workspace/models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8-300-10000/")

#**Training the Model on Efficientdet d1**

In [ ]:
!python /content/gdrive/MyDrive/Tensorflow/workspace/model_main_tf2.py --model_dir=$dst --pipeline_config_path=$fpath

#**Exporting the Model**

In [ ]:
!python /content/gdrive/MyDrive/Tensorflow/workspace/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=$fpath --trained_checkpoint_dir=$dst --output_directory=/content/gdrive/MyDrive/Tensorflow/workspace/exported_models/$model_name

#**Setting things for Faster-RCNN Incepetion Resnet**

In [ ]:
path = "/content/gdrive/MyDrive/Tensorflow/workspace/pretrained_models"
fname = "pipeline.config"
model_name = "faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8"
src = "/content/gdrive/MyDrive/Tensorflow/workspace/pretrained_models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config"
dst = "/content/gdrive/MyDrive/Tensorflow/workspace/models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/"
path_label = "/content/gdrive/MyDrive/Tensorflow/workspace/LabelMap.pbtxt" 
annot = "/content/gdrive/MyDrive/Tensorflow/workspace/annotations"
fpath = os.path.join("/content/gdrive/MyDrive/Tensorflow/workspace/models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8", fname)
LabelMap = map_util.create_category_index_from_labelmap(
    path_label, 
    use_display_name = True
)

#**Downloading Faster RCNN Inception Resnet**

In [ ]:
url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz"

r = requests.get(url)

# Extract model
thetarfile = tarfile.open(
    fileobj = BytesIO(r.content), 
    mode = "r|gz"
)

# Save model
thetarfile.extractall(path = path)

In [ ]:
# Moving pipeline.config file to models directory

os.makedirs(dst, exist_ok = True)

copy2(src, dst)

#**Editing pipeline.config for Faster RCNN Inception Resnet**

In [ ]:
# Making recommended changes
config_dic = config_util.get_configs_from_pipeline_file(fpath)

config_dic["model"].faster_rcnn.num_classes = len(LabelMap)
config_dic["model"].faster_rcnn.image_resizer.keep_aspect_ratio_resizer.min_dimension = 100
config_dic["model"].faster_rcnn.image_resizer.keep_aspect_ratio_resizer.max_dimension = 512
config_dic["model"].faster_rcnn.image_resizer.keep_aspect_ratio_resizer.pad_to_max_dimension = False 


config_dic["train_config"].batch_size = 8
config_dic["train_config"].fine_tune_checkpoint = os.path.join("/content/gdrive/MyDrive/Tensorflow/workspace/pretrained_models", model_name, "checkpoint/ckpt-0")
config_dic["train_config"].fine_tune_checkpoint_type = "detection"
config_dic["train_config"].use_bfloat16 = False # Set to True if training on a TPU
config_dic["train_config"].num_steps = 15000

config_dic["train_input_config"].label_map_path = path_label
config_dic["train_input_config"].tf_record_input_reader.input_path[:] = train_data

config_dic["eval_input_configs"][0].label_map_path = path_label
config_dic["eval_input_configs"][0].tf_record_input_reader.input_path[:] = valid_data

In [ ]:
config = config_util.create_pipeline_proto_from_configs(config_dic)
config_util.save_pipeline_config(config, "/content/gdrive/MyDrive/Tensorflow/workspace/models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8-300-10000/")

#**Training the Model on Efficientdet d1**

In [ ]:
!python /content/gdrive/MyDrive/Tensorflow/workspace/model_main_tf2.py --model_dir=$dst --pipeline_config_path=$fpath

#**Exporting the Model**

In [ ]:
!python /content/gdrive/MyDrive/Tensorflow/workspace/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=$fpath --trained_checkpoint_dir=$dst --output_directory=/content/gdrive/MyDrive/Tensorflow/workspace/exported_models/$model_name

#**Creating TF Record for Test Data**

In [ ]:
def create_tf_record_test(img_path, max_dim):
    
    filename = img_path.split("/")[-1].encode()
    source_id = img_path.encode()
    
    # Preprocess image 
    img, h, w = read_dicom(img_path, max_dim)
    #height, width, _ = img.shape
    img = CLAHE(img)
    
    # Encode as JPEG (Lossy compression)
    img = tf.io.encode_jpeg(
        img, 
        quality = 100, 
        format = 'grayscale'
    )
    
    img_bytes = img.numpy()
    
    img_format = b'jpeg'

    
    # Creating TFRecord
    tf_record = tf.train.Example(
        features = tf.train.Features(
            feature = {
                'image/height': dataset_util.int64_feature(h),
                'image/width': dataset_util.int64_feature(w),
                'image/filename': dataset_util.bytes_feature(filename),
                'image/source_id': dataset_util.bytes_feature(source_id),
                'image/encoded': dataset_util.bytes_feature(img_bytes),
                'image/format': dataset_util.bytes_feature(img_format),
            }
        )
    )
    
    return tf_record

In [ ]:
annot_path = "/kaggle/working/workspace1/test-annotations" 
os.makedirs(annot_path, exist_ok = True) 

In [ ]:
num_shards=1
img_cnt = np.zeros(num_shards, dtype = int)

filenames=os.listdir('../input/vinbigdata-chest-xray-abnormalities-detection/test')
with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
        tf_record_close_stack, 
        annot_path, 
        num_shards
    )
    
    for i in range(num_shards):
    #df_shard = df[df["fold"] == i]
        xrays = set(filenames[:])

        for xray in tqdm(xrays):
            #df_image = df_shard[df_shard["image_id"] == xray]

            img_path = os.path.join('../input/vinbigdata-chest-xray-abnormalities-detection/test', xray)
            tf_record = create_tf_record_test(img_path, 500)
            output_tfrecords[i].write(tf_record.SerializeToString())
            
        img_cnt[i] += 1

print("Converted {} images".format(np.sum(img_cnt)))
print("Images per shard: {}".format(img_cnt))
#print('DONE')

#**TESTING**

In [ ]:
test_data=[]
test_data.append('/content/gdrive/MyDrive/Tensorflow/workspace/test-annotations/test-annotations-00000-of-00001') 
test_data[:1]

In [ ]:
for shard in test_data[:1]:
    raw_dataset = tf.data.TFRecordDataset(shard)
    print(raw_dataset)
images=[]
original_dimension=[]
filenames=[]
for raw_record in tqdm(raw_dataset.take(3000)): # Select one shard from the TFRecords dataset
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    img_encoded = example.features.feature['image/encoded'].bytes_list.value[0]
    img = tf.io.decode_jpeg(img_encoded)
    #print(img.shape)
    img = cv2.cvtColor(img.numpy(), cv2.COLOR_GRAY2RGB)
    img = img[tf.newaxis, ...]
    img_width = example.features.feature['image/width'].int64_list.value[0]
    img_height = example.features.feature['image/height'].int64_list.value[0]
    filename = example.features.feature['image/filename'].bytes_list.value[0]
    filename = str(filename,'utf-8')
    filename=filename.split(".")[0]
    filenames.append(filename)
    original_dimension.append(tuple((img_width,img_height)))
    images.append(img)
#print(example)

In [ ]:
predictions

In [ ]:
df = {'image_id':filenames,'PredictionString':predictions}
df = pd.DataFrame(df)
print(df)
df = df.sort_values(by=['image_id'])
df.to_csv('submission.csv',index = False)

#**References**

https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b

https://medium.com/swlh/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc

https://www.youtube.com/watch?v=COlbP62-B-U&list=PLQVvvaa0QuDcNK5GeCQnxYnSSaar2tpku

https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

https://www.kaggle.com/bhallaakshit/eda-and-preprocessing-for-tf2-object-detection

https://www.kaggle.com/bhallaakshit/dicom-training-prediction-and-evaluation